In [9]:
from itertools import permutations
import binascii

def text_to_binary(text):
    return ''.join(format(ord(c), '08b') for c in text)

def binary_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join(chr(int(c, 2)) for c in chars)

def initial_permutation(text):
    return text[::-1]  # Just an example, not actual DES IP

def final_permutation(text):
    return text[::-1]  # Reverse of initial permutation

def generate_subkeys(key):
    subkeys = [key[i:] + key[:i] for i in range(16)]  # Simplified shifting
    return subkeys

def expansion_function(block):
    return block + block[:4]  # Simplified expansion

def s_box_substitution(block):
    return ''.join(str((int(c) + 1) % 2) for c in block)  # Flip bits as example

def p_box_permutation(block):
    return block[::-1]  # Reverse as a placeholder for actual permutation

def feistel_round(left, right, key):
    expanded_right = expansion_function(right)
    xor_result = ''.join(str(int(a) ^ int(b)) for a, b in zip(expanded_right, key))
    substituted = s_box_substitution(xor_result)
    permuted = p_box_permutation(substituted)
    new_right = ''.join(str(int(a) ^ int(b)) for a, b in zip(left, permuted))
    return right, new_right  # Swapping left and right

def des_encrypt(plaintext, key):
    binary_plaintext = text_to_binary(plaintext)
    binary_key = text_to_binary(key)
    binary_key = (binary_key * ((len(binary_plaintext) // len(binary_key)) + 1))[:len(binary_plaintext)]  # Repeat key if needed
    
    binary_plaintext = initial_permutation(binary_plaintext)
    left, right = binary_plaintext[:len(binary_plaintext)//2], binary_plaintext[len(binary_plaintext)//2:]
    subkeys = generate_subkeys(binary_key)
    for i in range(16):
        left, right = feistel_round(left, right, subkeys[i])
    ciphertext = final_permutation(right + left)  # Swap back before final permutation
    return ciphertext

def des_decrypt(ciphertext, key):
    binary_key = text_to_binary(key)
    binary_key = (binary_key * ((len(ciphertext) // len(binary_key)) + 1))[:len(ciphertext)]  # Repeat key if needed
    
    ciphertext = initial_permutation(ciphertext)  # Reverse final permutation
    left, right = ciphertext[:len(ciphertext)//2], ciphertext[len(ciphertext)//2:]
    subkeys = generate_subkeys(binary_key)[::-1]  # Reverse subkeys for decryption
    for i in range(16):
        left, right = feistel_round(left, right, subkeys[i])
    decrypted_binary = final_permutation(right + left)  # Swap back before final permutation
    return binary_to_text(decrypted_binary)

# Example usage
plaintext = "ellatieneunahermana"  # Example text
key = "medico"  # Example key
ciphertext = des_encrypt(plaintext, key)
print("Ciphertext (binary):", ciphertext)

decrypted_text = des_decrypt(ciphertext, key)
print("Decrypted text:", decrypted_text)


Ciphertext (binary): 11000010001101110111101011011001001001110011001101010010011101110111101000010011100101011011000001010101000101000110010010110011011001010110000011011000
Decrypted text: ellatieneunahermana
